In [247]:
search_company = 'eris'

In [248]:
# company search
import requests
import json

headers = headers_dict = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/api/v1/ind/company_search/?q={search_company}", 
    headers=headers
)

search_results = []
slug = ''

if not 200 <= response.status_code < 400:
    print(response.text)
else:
    for item in json.loads(response.text):
        if item['type'] == 'companies':
            search_results.append(
                {
                    'name' : item['name'],
                    'slug' : item['slug']
                }
            )
    if len(search_results) > 1:
        print(json.dumps(search_results, ensure_ascii=False, indent=4))
    elif len(search_results) == 1:
        slug = search_results[0]['slug']
        print("Single slug found and loaded, please skip next cell and proceed")
    else:
        print("No matching company found")

[
    {
        "name": "Amerise Biosciences Ltd.",
        "slug": "amradeep-industries-ltd"
    },
    {
        "name": "Eris Lifesciences Ltd.",
        "slug": "eris-lifesciences"
    }
]


In [249]:
slug = 'eris-lifesciences'

In [250]:
# fetch particular company info

headers = headers_dict = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response_overview = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}", 
    headers=headers
)

if not 200 <= response_overview.status_code < 400:
    print(response_overview.text)
else:
    print(f"Success fetch overview - response_overview -> https://www.tijorifinance.com/company/{slug}")

response_financials = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/financials/", 
    headers=headers
)

if not 200 <= response_financials.status_code < 400:
    print(response_financials.text)
else:
    print(f"Success fetch financials - response_financials -> https://www.tijorifinance.com/company/{slug}/financials/")

response_benchmarking = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/benchmarking/", 
    headers=headers
)

if not 200 <= response_benchmarking.status_code < 400:
    print(response_benchmarking.text)
else:
    print(f"Success fetch benchamarking - response_benchmarking -> https://www.tijorifinance.com/company/{slug}/benchmarking/")
    
response_shareholding = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/shareholding/", 
    headers=headers
)

if not 200 <= response_shareholding.status_code < 400:
    print(response_shareholding.text)
else:
    print(f"Success fetch shareholding - response_shareholding -> https://www.tijorifinance.com/company/{slug}/shareholding/")

Success fetch overview - response_overview -> https://www.tijorifinance.com/company/eris-lifesciences
Success fetch financials - response_financials -> https://www.tijorifinance.com/company/eris-lifesciences/financials/
Success fetch benchamarking - response_benchmarking -> https://www.tijorifinance.com/company/eris-lifesciences/benchmarking/
Success fetch shareholding - response_shareholding -> https://www.tijorifinance.com/company/eris-lifesciences/shareholding/


In [251]:
# Get about, market cap, pe ratio

soup = BeautifulSoup(response_overview.text, 'html.parser')

data = {}
data['summary'] = soup.find('div', class_='about_desc detailed_desc').text.strip().split('\n')[0].strip()
data['market_cap'] = (int((soup.find('div', string='Market Cap').find_next('div', class_='company_details_value').text).replace('₹', '').replace(' Cr.', '').replace(',', '')) * 10**7 )
data['pe_ratio'] = float(soup.find('div', string='P/E').find_next('div', class_='company_details_value').text)


print(json.dumps(data, ensure_ascii=False, indent=4))

{
    "summary": "Eris Lifesciences Limited, founded in 2007 by Amit Bakshi, is a rapidly growing company in the Indian Branded Formulations market. They specialize in developing, manufacturing, and commercializing pharmaceutical products in therapeutic areas like cardiovascular, anti-diabetes, vitamins, gastroenterology, and gynaecology, focusing on lifestyle-related disorders.",
    "market_cap": 189700000000,
    "pe_ratio": 49.88
}


In [252]:
# Get all FAQs list

from bs4 import BeautifulSoup
import json

faq_data = []

for html_code in [response_overview, response_financials, response_benchmarking, response_shareholding]:
    soup = BeautifulSoup(html_code.text, 'html.parser')
    
    faq_containers = soup.find_all('div', class_='faq_container')
    
    for container in faq_containers:
        question = container.find('div', class_='faq_data_header').find('span').text.strip()
        answer = container.find('div', class_='faq_answer').find('p').text.strip()
        
        faq_data.append(
            {
                'question': question,
                'answer': answer
            }
        )
    
print(json.dumps(faq_data, ensure_ascii=False, indent=4))

[
    {
        "question": "What does Eris Lifesciences do?",
        "answer": "Eris Lifesciences Limited, founded in 2007, is a fast-growing company in the Indian Branded Formulations market, focusing on cardiovascular, anti-diabetes, vitamins, gastroenterology, and gynecology products."
    },
    {
        "question": "Who are the competitors of Eris Lifesciences?",
        "answer": "Eris Lifesciences major competitors are Jubilant Pharmova, Astrazeneca Pharma I, Sanofi India, Caplin Point Lab, Wockhardt, Strides Pharma Scien, Marksans Pharma. Market Cap of Eris Lifesciences is ₹18,392 Crs. While the median market cap of its peers are ₹14,955 Crs."
    },
    {
        "question": "Is Eris Lifesciences financially stable compared to its competitors?",
        "answer": "Eris Lifesciences seems to be less financially stable compared to its competitors.Altman Z score of Eris Lifesciences is 3.62 and is ranked 6 out of its 8 competitors."
    },
    {
        "question": "Has the sh

In [253]:
# Get company forensics ( 1 - good, 2 - neutral, 3 - bad, 4 - not calculated )
soup = BeautifulSoup(response_overview.text, 'html.parser')

script_data = soup.find('script', id='company_details_data', type='application/json').text.strip()
forensics_data = json.loads(script_data)

if forensics_data['quick_look']:
    # Collect the factories
    forensics = forensics_data['quick_look']['data'][0]['factories']
    forensics.extend(forensics_data['quick_look']['data'][1]['factories'])
    
    forensics = sorted(forensics, key=lambda x: x['flag'], reverse=True)
    
    print(json.dumps(forensics, ensure_ascii=False, indent=4))
else:
    print('No forensics data available')

[
    {
        "name": "Revenue Recognition",
        "sentence": "Chances of aggressive revenue recognition - operating profit of company is not getting converted to cash and recievables rising faster than sales",
        "explanation": "A company might stuff the channel or book dummy sales. We check for quality of these sales and if profits are getting converted to cash by combing the P&L and Cash Flow statement.",
        "flag": 3
    },
    {
        "name": "Current vs Historic Valuation",
        "sentence": "Company is trading at a premium to 3yr historic valuations",
        "explanation": "Current valuation metrics like PE, Price to book, EV by EBITDA etc are looed at with respect to historic values and a score is given to gauge over valuation or under valuation\n\n",
        "flag": 3
    },
    {
        "name": "ROE",
        "sentence": "Roe has decreased vs 10 year trend",
        "explanation": "We check if the driving factor behind ROE movement is margins, leverage or

In [254]:
# Get peers data as data frame

import pandas as pd
from IPython.display import HTML

def display_scrollable_dataframe(df_pivot):
    html = df_pivot.to_html(classes='table table-bordered', index=False)
    return HTML(f"""
    <div style="height:300px; overflow:auto; border:1px solid lightgrey; padding:5px;">
    {html}
    </div>
    """)

soup = BeautifulSoup(response_overview.text, 'html.parser')

script_data = soup.find('script', id='peers_table_data', type='application/json').text.strip()
peers_data = json.loads(script_data)

df_peers_data = pd.DataFrame(peers_data[1]['data'])

display_scrollable_dataframe(df_peers_data)

Latest Price,PE,Market Capital,Promoter,YoY Qtly Net Sales,ROCE (%),ROE (%),name,slug
1390.00,49.88,18970.0,54.88,54.24,12.258009,15.157191,Eris Lifesciences,eris-lifesciences
7742.40,201.93,19356.0,75.00,31.15,25.231244,22.686538,Astrazeneca Pharma I,astrazeneca-pharma-india-limited
1183.80,34.11,18856.0,50.68,9.12,6.233292,1.418870,Jubilant Pharmova,jubilant-life-sciences-limited
6882.55,30.0,15883.0,60.40,-9.98,38.254693,59.387313,Sanofi India,sanofi-india-limited
947.05,0.00,14537.0,51.99,14.75,-2.360365,-13.783864,Wockhardt,wockhardt-limited
1190.00,37.15,23404.0,69.61,5.09,14.291582,12.781122,Alembic Pharma,alembic-pharmaceuticals-limited
1888.70,30.06,14356.0,70.56,16.10,23.996619,19.738655,Caplin Point Lab,caplin-point-laboratories-limited
1513.00,0.00,13945.0,25.86,16.94,20.878228,-3.322132,Strides Pharma Scien,strides-pharma-science-limited


In [255]:
# Shareholding data as dataframe

soup = BeautifulSoup(response_shareholding.text, 'html.parser')

all_data = []

for script_tag in soup.find_all('script'):
    js_code = script_tag.string
    if js_code: 
        try:
            if 'var trendData' in js_code or 'var trendData2' in js_code:
                start_index = js_code.index('[') 
                end_index = js_code.rindex(']') + 1

                json_string = js_code[start_index:end_index]
                data = json.loads(json_string.replace("'", '"')) 

                all_data.append(data)
        except Exception as e:
            print(f"Error extracting JSON from script: {e}")

flattened_data = []
for dataset in all_data:
    for entry in dataset:
        name = entry['name']
        for item in entry['data']:
            timestamp = item[0]  # The timestamp
            value = item[1]      # The associated value
            flattened_data.append({'Name': name, 'Timestamp': timestamp, 'Value': value})



df = pd.DataFrame(flattened_data)
df['DateIST'] = pd.to_datetime(df['Timestamp'], unit='ms') + pd.Timedelta(hours=5, minutes=30)
df['DateIST'] = df['DateIST'].dt.strftime('%Y-%m')

df_shareholder_info = df.pivot_table(index='Name', columns='DateIST', values='Value')
df_shareholder_info.reset_index(inplace=True)

display_scrollable_dataframe(df_shareholder_info)

DateIST,Name,2016-10,2017-06,2017-09,2017-12,2018-03,2018-06,2018-09,2018-12,2019-03,2019-06,2019-09,2019-12,2020-03,2020-06,2020-09,2020-12,2021-03,2021-06,2021-09,2021-12,2022-03,2022-06,2022-09,2022-12,2023-03,2023-06,2023-09,2023-12,2024-03,2024-06
,DII-Others,0.00,0.95,0.72,0.45,0.45,0.45,0.46,0.50,0.59,0.59,0.59,0.67,0.66,0.68,0.56,0.58,0.41,0.39,0.37,0.38,0.40,0.31,0.31,0.30,0.38,0.45,0.66,0.70,0.71,0.81
,FII,0.00,7.18,6.12,6.76,8.59,8.68,8.65,8.42,8.14,7.92,6.06,8.85,10.19,10.11,11.13,11.30,12.42,13.49,13.60,13.34,13.31,15.35,15.39,15.50,14.86,13.78,13.20,13.13,14.27,14.59
,Individual < 2 lac,0.00,1.84,1.77,1.36,1.32,1.21,1.19,1.14,1.07,1.08,1.18,1.36,1.04,1.25,1.85,2.07,1.74,1.80,1.91,1.82,1.77,1.83,1.76,1.95,2.16,2.17,2.29,2.53,2.68,2.47
,Insurance Companies,0.00,0.55,0.60,0.51,0.07,0.05,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.15,0.05,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.90
,Mutual Funds,0.00,8.71,11.17,11.44,10.29,10.29,10.69,10.85,11.05,11.12,12.84,11.13,10.51,10.66,10.19,9.63,9.23,9.66,9.57,9.99,10.11,10.36,10.38,9.64,9.64,10.27,13.86,13.85,14.92,14.52
,Others,40.82,24.84,23.69,23.55,23.35,23.39,23.08,23.05,23.05,23.06,23.10,21.75,22.05,21.75,22.04,22.14,22.05,21.97,21.87,21.81,21.75,19.85,19.87,19.91,20.10,9.09,4.13,3.94,0.00,0.00
,Promoter,59.18,55.93,55.93,55.93,55.93,55.93,55.92,56.04,56.10,56.23,56.23,56.23,55.55,55.55,54.08,54.23,54.13,52.69,52.68,52.66,52.66,52.30,52.29,52.70,52.86,52.86,54.91,54.90,54.90,54.88
,Shares Pledged by Promoters(As a % of Total Promoter Holding),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.37,10.95,10.95,18.51,18.51
